First install the Squall repository from github and place this notebook in the directory

# IMPORTS

In [121]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Squall Dataset

In [122]:
# Read the JSON file
with open('data/squall.json', 'r') as f:
    data = json.load(f)

# Convert JSON to DataFrame
df = pd.DataFrame(data)

In [123]:
# Different columns of the squall dataset (More info about the columns can be found at: https://github.com/tzshi/squall)
print(df.columns)

Index(['nt', 'columns', 'nl', 'nl_pos', 'nl_ner', 'nl_ralign', 'nl_typebio',
       'nl_typebio_col', 'nl_incolumns', 'nl_incells', 'columns_innl', 'tgt',
       'tbl', 'sql', 'align'],
      dtype='object')


In [124]:
# First 5 table ids
print(df['tbl'].head(5))

0    203_447
1    203_447
2    203_447
3    203_447
4    203_447
Name: tbl, dtype: object


# PREPROCESSING SQUALL DATASET

Since the SQL queries and natural language questions are formatted as nested lists we need to concatenate them to make them more readable/easier to process

In [125]:
def flatten_sql(sql_tokens):
    return " ".join([token[1] for token in sql_tokens])

# Adds new 'flattened_sql column to the dataset
df['flattened_sql'] = df['sql'].apply(flatten_sql)

In [126]:
# Set pandas display option to show full column width
pd.set_option('display.max_colwidth', None)

# First five SQL queries
print(df['flattened_sql'].head(5))

0    select ( select c5_number from w where c1_number = 1 ) - ( select c5_number from w where c1_number = 2 )
1                                                                         select count ( distinct c1 ) from w
2          select c2 from w where id = ( select id from w where c2 = 'atacama' order by id desc limit 1 ) + 1
3                                                                select c4 from w where c3 = 'jaime quintana'
4                                                                   select c2 from w order by id desc limit 1
Name: flattened_sql, dtype: object


We do the same for the natural language questions

In [127]:
def flatten_nl(nl_tokens):
    # Join the tokens into a single string
    return " ".join(nl_tokens)

# Apply the function to the 'nl' column
df['flattened_nl'] = df['nl'].apply(flatten_nl)


In [128]:
# First five SQL queries
print(df['flattened_nl'].head(5))

0    what is the difference in years between constiuency 1 and 2 ?
1                     what is the total number of constituencies ?
2                           which region is listed below atacama ?
3                       which party did jaime quintana belong to ?
4                    what is the last region listed on the table ?
Name: flattened_nl, dtype: object


# CREATING OPERATOR DATASETS

We now filter the dataset to obtain datasets for each SQL operator where we do not allow multiple (mathematical) SQL operators to be in the same query

In [129]:
sql_operators = {
    'sum', 'max', 'avg', 'min', 'count',   
}

# Initialize a dictionary to store DataFrames for each operator
operator_dfs = {}

# Iterate through each operator and create a DataFrame for each
for operator in sql_operators:
    # Filter the original DataFrame for rows containing the operator in 'flattened_sql'
    operator_df = df[df['flattened_sql'].str.contains(r'\b' + operator + r'\b', case=False, na=False)]
    
    # Further filter to ensure that the query contains only this operator
    # This is done by checking that no other operators are present in the query
    # for other_operator in sql_operators - {operator}:
    #     operator_df = operator_df[~operator_df['flattened_sql'].str.contains(r'\b' + other_operator + r'\b', case=False, na=False)]
    
    # Store the DataFrame in the dictionary with the operator as the key
    operator_dfs[operator] = operator_df

# Print the number of rows for each operator DataFrame
for operator, operator_df in operator_dfs.items():
    print(f"DataFrame for operator '{operator}': {len(operator_df)} rows")

DataFrame for operator 'count': 3728 rows
DataFrame for operator 'sum': 279 rows
DataFrame for operator 'min': 210 rows
DataFrame for operator 'avg': 51 rows
DataFrame for operator 'max': 234 rows


# Matching with the SQL dataset

Now copy the pristine-unseen-tables.tsv and random-split-1-dev.tsv files from the SQA dataset and put them in the squall directory. These are the test and validation sets for SQA used in TAPEX 

We load the dev and test files into dataframes

In [130]:
import pandas as pd

# Load the TSV file into a DataFrame, skipping bad lines
validation_file_path = 'random-split-1-dev.tsv'  # Adjust the path as necessary
val_df = pd.read_csv(validation_file_path, sep='\t', on_bad_lines='skip')
print(f"Validation DataFrame rows: {val_df.shape[0]}")

test_file_path = 'pristine-unseen-tables.tsv'  # Adjust the path as necessary
test_df = pd.read_csv(test_file_path, sep='\t')
print(f"Test DataFrame rows: {val_df.shape[0]}")

train_file_path = 'random-split-1-train.tsv'  # Adjust the path as necessary
train_df = pd.read_csv(train_file_path, sep='\t')
print(f"Test DataFrame rows: {train_df.shape[0]}")

Validation DataFrame rows: 2831
Test DataFrame rows: 2831
Test DataFrame rows: 11321


In [131]:
# Different columns of the datasets
print(test_df.columns)

Index(['id', 'utterance', 'context', 'targetValue'], dtype='object')


In [132]:
# Printing the first 5 table id's
print(test_df['context'].head(5))

0    csv/203-csv/733.csv
1    csv/204-csv/149.csv
2    csv/203-csv/435.csv
3    csv/204-csv/803.csv
4    csv/204-csv/272.csv
Name: context, dtype: object


Since the table id's are different from the squall dataset we create a new id column that uses the same id format as squall

In [133]:
test_df['tbl'] = test_df['context'].str.replace('csv/', '').str.replace('.csv', '').str.replace('-', '_', regex=False)
val_df['tbl'] = val_df['context'].str.replace('csv/', '').str.replace('.csv', '').str.replace('-', '_', regex=False)
train_df['tbl'] = train_df['context'].str.replace('csv/', '').str.replace('.csv', '').str.replace('-', '_', regex=False)

# Printing the first 5 table id's
print(test_df['tbl'].head(5))

0    203_733
1    204_149
2    203_435
3    204_803
4    204_272
Name: tbl, dtype: object


Let's check for a random operator, the number of matches with the dev, train or test set

In [134]:
# Select a specific operator DataFrame (for example, 'sum')
operator = 'count'  # Change this to the operator you want to analyze
operator_df = operator_dfs[operator]  # Assuming operator_dfs is already defined

# Lowercase the 'flattened_nl' column for comparison
operator_df['flattened_nl_lower'] = operator_df['flattened_nl'].str.lower()

# Lowercase the 'utterance' columns in both test and validation DataFrames
test_df['utterance_lower'] = test_df['utterance'].str.lower()
val_df['utterance_lower'] = val_df['utterance'].str.lower()
train_df['utterance_lower'] = train_df['utterance'].str.lower()


# Compare the 'flattened_nl' column with the 'utterance' column in the test DataFrame
matches_test = operator_df[operator_df['flattened_nl_lower'].isin(test_df['utterance_lower'])]

# Print the results for the test DataFrame
print(f"Matching 'flattened_nl' values for operator '{operator}' in test set:")
print(matches_test[['flattened_nl']])  # Adjust the columns to display as needed
print(f"Number of matches in test set: {len(matches_test)}")

# Compare the 'flattened_nl' column with the 'utterance' column in the validation DataFrame
matches_val = operator_df[operator_df['flattened_nl_lower'].isin(val_df['utterance_lower'])]

# Print the results for the validation DataFrame
print(f"Matching 'flattened_nl' values for operator '{operator}' in validation set:")
print(matches_val[['flattened_nl']])  # Adjust the columns to display as needed
print(f"Number of matches in validation set: {len(matches_val)}")

# Compare the 'flattened_nl' column with the 'utterance' column in the validation DataFrame
matches_train = operator_df[operator_df['flattened_nl_lower'].isin(train_df['utterance_lower'])]

# Print the results for the validation DataFrame
print(f"Matching 'flattened_nl' values for operator '{operator}' in validation set:")
print(matches_train[['flattened_nl']])  # Adjust the columns to display as needed
print(f"Number of matches in validation set: {len(matches_train)}")

Matching 'flattened_nl' values for operator 'count' in test set:
Empty DataFrame
Columns: [flattened_nl]
Index: []
Number of matches in test set: 0
Matching 'flattened_nl' values for operator 'count' in validation set:
                                                        flattened_nl
387                                   number of teams above 9 medals
1720                                 how many dates are on the chart
1722   in how many games were than more than 80,000 people attending
2345           total number of members from lawton and oklahoma city
2657                                            total number of wins
3071                                            total number of wins
4068                   number of different teams listed on the chart
4209                            what is the total years on the chart
5008                 what is the number of titles ken harden has one
5213                                     how many gold did u.s.a win
7626                  

/var/folders/s_/1tg6zl197tzbfn_czq7vd7sw0000gn/T/ipykernel_21984/3094609716.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  operator_df['flattened_nl_lower'] = operator_df['flattened_nl'].str.lower()


Let's try it with question id's instead

In [135]:
# Adding nt column to data partitions

sql_operators = {
    'sum', 'max', 'avg', 'min', 'count',   
}

test_df['nt'] = test_df['id']
val_df['nt'] = val_df['id']
train_df['nt'] = train_df['id']

operator_dfs_filtered = {}

for operator in sql_operators: 
    
    print(f"operator: {operator}")
    
    operator_df = operator_dfs[operator]
    print(f"operator count in squall dataset: {operator_df.shape[0]}")


    # Checking matching id's with count df
    # id_matches_train = operator_df[operator_df['nt'].isin(train_df['nt'])]
    # id_matches_test = operator_df[operator_df['nt'].isin(test_df['nt'])]
    id_matches_val = val_df[val_df['nt'].isin(operator_df['nt'])]
    
    operator_df_filtered = val_df[val_df['nt'].isin(operator_df['nt'])]
    operator_dfs_filtered[operator] = operator_df_filtered
        
    # operator_dfs_filtered[operator] = operator_df.. 

    # Printing the number of matches in each set
    # print(f"Number of matches in train set: {len(id_matches_train)}")
    # print(f"Number of matches in test set: {len(id_matches_test)}")
    print(f"Number of matches in validation set: {len(id_matches_val)}")
    # print("-----------------------------------")


operator: count
operator count in squall dataset: 3728
Number of matches in validation set: 711
operator: sum
operator count in squall dataset: 279
Number of matches in validation set: 57
operator: min
operator count in squall dataset: 210
Number of matches in validation set: 30
operator: avg
operator count in squall dataset: 51
Number of matches in validation set: 10
operator: max
operator count in squall dataset: 234
Number of matches in validation set: 36


In [120]:
columns_to_drop = ['tbl', 'utterance_lower', 'nt']  # Replace with the actual columns you want to drop

for operator in sql_operators: 
    filtered_df = operator_dfs_filtered[operator].drop(columns=columns_to_drop, errors='ignore')
    filtered_df.to_csv(f'filtered_datasets/{operator}_filtered_split-1-dev.tsv', sep='\t', index=False)


In [118]:
import pandas as pd

# Function to filter examples
def filter_examples_by_ids(example_file, id_list, output_file):
    filtered_lines = []
    
    with open(example_file, 'r') as file:
        for line in file:
            if any(f"(id {id_})" in line for id_ in id_list):
                filtered_lines.append(line)
    
    # Write the filtered lines to the new file
    with open(output_file, 'w') as file:
        file.writelines(filtered_lines)
    
    print(f"Filtered examples saved to {output_file}")

# Path to your original .examples file
original_examples_file = 'random-split-1-dev.examples'

# Iterate over each operator and filter examples
for operator, operator_df in operator_dfs_filtered.items():
    id_list = operator_df['nt'].tolist()  # Get the list of IDs from the DataFrame
    output_file = f"filtered_datasets/{operator}_filtered_split-1-dev.examples"  # Output file name
    
    # Filter the .examples file and save
    filter_examples_by_ids(original_examples_file, id_list, output_file)


Filtered examples saved to filtered_datasets/count_filtered_split-1-dev.examples
Filtered examples saved to filtered_datasets/sum_filtered_split-1-dev.examples
Filtered examples saved to filtered_datasets/min_filtered_split-1-dev.examples
Filtered examples saved to filtered_datasets/avg_filtered_split-1-dev.examples
Filtered examples saved to filtered_datasets/max_filtered_split-1-dev.examples


In [102]:
operator_dfs_filtered['avg']

,nt,columns,nl,nl_pos,nl_ner,nl_ralign,nl_typebio,nl_typebio_col,nl_incolumns,nl_incells,columns_innl,tgt,tbl,sql,align,flattened_sql,flattened_nl
2542,nt-2263,"[[year, [year], [number], number, 1997], [single, [single], [first, second], text(text), ""dopeman"" (remix)], [peak chart positions\nus mod, [peak, chart, positions, us, mod], [number], number, 39], [peak chart positions\nuk, [peak, chart, positions, uk], [number], number, 51], [album, [album], [], text, losing streak]]","[what, was, the, average, chart, position, of, their, singles, in, the, uk, ?]","[WP, VBD-AUX, DT, JJ, NN, NN, IN, PRP$, NNS, IN, DT, NNP, .]","[O, O, O, O, O, O, O, O, O, O, O, LOCATION, O]","[[None, None], [None, None], [None, None], [Keyword, [agg, agg_avg]], [Column, c4_number], [Column, c4_number], [None, None], [None, None], [None, None], [Column, c4_number], [Column, c4_number], [Column, c4_number], [None, None]]","[O, O, O, B-Keyword, B-Column, I-Column, O, O, O, B-Column, I-Column, I-Column, O]","[None, None, None, None, c4, c4, None, None, None, c4, c4, c4, None]","[False, False, False, False, True, False, False, False, False, False, False, True, False]","[False, False, True, False, False, False, True, False, False, True, True, False, True]","[False, False, True, True, False]",60.5,203_661,"[[Keyword, select, []], [Keyword, avg, []], [Keyword, (, []], [Column, c4_number, []], [Keyword, ), []], [Keyword, from, []], [Keyword, w, []]]","[[[3], [1, 4, 2]], [[4, 5, 9, 10, 11], [3]]]",select avg ( c4_number ) from w,what was the average chart position of their singles in the uk ?
4873,nt-1569,"[[name, [name], [], text, ned barkas], [nation, [nation], [], text, england], [position, [position], [], text, df], [league apps, [league, apps], [number], number, 4], [league goals, [league, goals], [number], number, 0], [fa cup apps, [fa, cup, apps], [number], number, 0], [fa cup goals, [fa, cup, goals], [number], number, 0], [total apps, [total, apps], [number], number, 4], [total goals, [total, goals], [number], number, 0]]","[what, is, the, average, number, of, scotland, 's, total, apps, ?]","[WP, VBD-AUX, DT, JJ, NN, IN, NNP, POS, JJ, NNS, .]","[O, O, O, O, O, O, LOCATION, O, O, O, O]","[[None, None], [None, None], [Keyword, [agg, agg_avg]], [Keyword, [agg, agg_avg]], [Keyword, [agg, agg_avg]], [Keyword, [agg, agg_avg]], [Literal, None], [None, None], [Column, c8_number], [Column, c8_number], [None, None]]","[O, O, B-Keyword, I-Keyword, I-Keyword, I-Keyword, B-String, O, B-Column, I-Column, O]","[None, None, None, None, None, None, c2, None, c8, c8, None]","[False, False, False, False, False, False, False, False, True, True, False]","[False, False, False, False, False, False, True, False, False, False, False]","[False, False, False, True, False, True, False, True, True]",15.5,204_159,"[[Keyword, select, []], [Keyword, avg, []], [Keyword, (, []], [Column, c8_number, []], [Keyword, ), []], [Keyword, from, []], [Keyword, w, []], [Keyword, where, []], [Column, c2, []], [Keyword, =, []], [Literal.String, 'scotland', [6, 6]]]","[[[3, 2, 4, 5], [1, 2, 4]], [[8, 9], [3]], [[6], [10]]]",select avg ( c8_number ) from w where c2 = 'scotland',what is the average number of scotland 's total apps ?
4879,nt-13040,"[[name, [name], [], text, ned barkas], [nation, [nation], [], text, england], [position, [position], [], text, df], [league apps, [league, apps], [number], number, 4], [league goals, [league, goals], [number], number, 0], [fa cup apps, [fa, cup, apps], [number], number, 0], [fa cup goals, [fa, cup, goals], [number], number, 0], [total apps, [total, apps], [number], number, 4], [total goals, [total, goals], [number], number, 0]]","[average, number, of, goals, scored, by, players, from, scotland]","[JJ, NN, IN, NNS, VBN, IN, NNS, IN, NNP]","[O, O, O, O, O, O, O, O, LOCATION]","[[Keyword, [agg, agg_avg]], [Keyword, [agg, agg_avg]], [Keyword, [agg, agg_avg]], [Column, c9_number], [None, None], [None, None], [None, None], [Column, c2], [Literal, None]]","[B-Key

# Finding matching IDs for different datasets

In [70]:
# test_df, val_df, df 
# formatted_context 
# print(test_df['tbl'].head())
# print(df['tbl'].head())


test_tables_set = set(test_df['tbl'])
val_tables_set = set(val_df['tbl'])
train_tables_set = set(train_df['tbl'])

squall_tables_set = set(df['tbl'])

print(f"Number of different table ids in squall_tables_set {len(squall_tables_set)}")

print(f"Number of different table ids in val_tables_set {len(val_tables_set)}")
print(f"Number of different table ids in test_tables_set {len(test_tables_set)}\n")
print(f"Number of different table ids in train_tables_set {len(train_tables_set)}\n")



matching_test_tables = squall_tables_set.intersection(test_tables_set)
matching_val_tables = squall_tables_set.intersection(val_tables_set)
matching_train_tables = squall_tables_set.intersection(train_tables_set)

print(f"Matching tables between Squall dataset and test set: {matching_test_tables}")
print(f"Number of matching tables in Squall dataset with test set: {len(matching_test_tables)}\n")

print(f"Matching tables between Squall dataset and validation set: {matching_val_tables}")
print(f"Number of matching tables in Squall dataset with validation set {len(matching_val_tables)}\n")

print(f"Matching tables between Squall dataset and train set: {matching_train_tables}")
print(f"Number of matching tables between Squall dataset and train set {len(matching_train_tables)}")

Number of different table ids in squall_tables_set 1617
Number of different table ids in val_tables_set 346
Number of different table ids in test_tables_set 421

Number of different table ids in train_tables_set 1333

Matching tables between Squall dataset and test set: set()
Number of matching tables in Squall dataset with test set: 0

Matching tables between Squall dataset and validation set: {'203_509', '203_213', '201_28', '204_966', '200_25', '204_600', '203_501', '203_337', '204_942', '204_634', '203_127', '202_22', '204_698', '204_105', '203_370', '202_32', '203_104', '204_704', '204_171', '202_115', '203_456', '203_343', '203_355', '203_105', '203_654', '203_783', '204_772', '204_784', '204_601', '204_626', '203_390', '204_168', '202_240', '204_976', '203_775', '204_965', '201_40', '204_453', '204_643', '204_205', '204_552', '204_74', '201_36', '204_752', '204_564', '204_159', '203_702', '204_235', '203_721', '204_834', '203_862', '204_75', '204_455', '203_313', '204_929', '204